#### 사용할 패키지 추가

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

#### 입지 선정 후보에 대한 시설명과 변수들이 저장된 최종 데이터파일 읽기

In [2]:
delivery_box = pd.read_csv("최종후보데이터.csv", header=0, encoding='cp949', engine='python')

# 읽어들인 csv 파일을 데이터프레임으로 변환하기
df_dBox = pd.DataFrame(delivery_box)

# 전체 컬럼명 이름
print("전체 컬럼명 : ")
list(df_dBox.columns)

전체 컬럼명 : 


['시설명',
 '시설주소',
 '경도',
 '위도',
 '시설종류상세명',
 '동이름',
 '일인가구비율',
 'cctv개수',
 '건물가점',
 '파출소거리역수값',
 '백미터반경주택수']

In [3]:
# 저장된 데이터프레임의 처음 다섯행 확인하기
df_dBox.head()

,시설명,시설주소,경도,위도,시설종류상세명,동이름,일인가구비율,cctv개수,건물가점,파출소거리역수값,백미터반경주택수
0,서울성모원,서울특별시 영등포구 대림로12가길 7-1,126.904639,37.491490,노인주거복지시설,영등포본동,0.197152,5,1,0.099977,139
1,구립영등포노인케어센터,서울특별시 영등포구 도림로 482,126.890343,37.516883,노인의료복지시설,대림2동,0.380264,2,2,0.132986,6
2,구립영등포노인종합복지관,서울특별시 영등포구 도림로 482,126.890343,37.516883,노인여가복지시설,대림2동,0.380264,2,2,0.132986,6
3,구립영등포노인종합복지관병설데이케어센터,서울특별시 영등포구 도림로 482,126.890343,37.516883,재가노인복지시설,대림2동,0.380264,2,2,0.132986,6
4,구립여의도원광데이케어센터,서울특별시 영등포구 여의대방로68길 15영창빌딩 3층,126.932894,37.517969,재가노인복지시설,대림2동,0.380264,4,0,0.045181,4


#### 점수화에 필요한 컬럼명만 저장 

In [4]:
col_name = list(df_dBox.columns[6:])
col_name

['일인가구비율', 'cctv개수', '건물가점', '파출소거리역수값', '백미터반경주택수']

In [5]:
len_weight = len(col_name)
print(len_weight)

5


#### R에서 ahp분석을 통해 구한 가중치가 저장된 파일 읽기

In [6]:
dBox_weight = pd.read_csv("atts_weight_dBox.csv", header=0, engine='python')
dBox_weight
df_dBox_weight = pd.DataFrame(dBox_weight)
df_dBox_weight

,weight,일인가구비,방범cctv,건물가점,파출소거리,백미터반경주택
0,mean,0.183283,0.122942,0.060721,0.236541,0.396512
1,sd,0.024315,0.015162,0.004472,0.036848,0.042077


#### 가중치 데이터프레임에서 값만 weight리스트에 저장

In [7]:
# 빈 리스트 생성
weight = []
# df_dBox_weight dataframe에서 가중치를 의미하는 모든 변수에 대한 mean값을 저장
for i in range(len_weight):
    weight.append(df_dBox_weight.iloc[0][i+1])
weight

[0.183283438, 0.122942006, 0.060721334, 0.236541204, 0.396512018]

#### 가중치 적용할 변수에 대해 minmax 정규화

In [8]:
norm_df_dBox = df_dBox.copy()
norm_df_dBox = norm_df_dBox.sort_values('시설명')
norm_df_dBox.head()

,시설명,시설주소,경도,위도,시설종류상세명,동이름,일인가구비율,cctv개수,건물가점,파출소거리역수값,백미터반경주택수
71,(주)감사하는사람들노인복지센터,서울특별시 영등포구 시흥대로 649301호 (대림동),126.902758,37.485598,재가노인복지시설,대림2동,0.380264,4,0,0.070019,3
72,21세기요양원 노인요양공동생활가정,서울특별시 영등포구 도신로69 4층층 (도림동),126.900090,37.506351,노인의료복지시설,도림동,0.380264,3,2,0.098262,49
52,Vine Center,서울특별시 영등포구 양평로12가길 8 (당산동6가),126.900249,37.536375,한부모가족복지시설,당산6동,0.380264,2,1,0.267272,1
76,가온실버복지센터,서울특별시 영등포구 여의대방로 379 8층 14-1호 (여의도동 제일빌딩),126.930932,37.519141,재가노인복지시설,여의동,0.278519,0,0,0.055300,0
96,가족사랑1,서울특별시 영등포구 도신로204 4층 (신길동),126.913932,37.511183,노인의료복지시설,신길동,0.278519,8,2,0.106648,93


In [9]:
# Null 이 있을 수 있는 열의 값은 Null을 '   -   '로 변경
norm_df_dBox.fillna('   -   ', inplace=True)

#### min-max normalization

In [10]:
min_max_scaler = MinMaxScaler()
fitted = min_max_scaler.fit(norm_df_dBox.loc[:,col_name])
print(fitted.data_max_)

[  0.58699101  11.           7.           6.4756929  149.        ]


In [11]:
norm_df_dBox[col_name] = min_max_scaler.transform(norm_df_dBox.loc[:, col_name])
#print(norm_df_dBox.head())
#norm_df_dBox = norm_df_dBox.drop(['index'], axis=1)
norm_df_dBox.reset_index(inplace=True)

#### 정규화된 결과 dataframe 출력 

In [12]:
norm_df_dBox.head()

,index,시설명,시설주소,경도,위도,시설종류상세명,동이름,일인가구비율,cctv개수,건물가점,파출소거리역수값,백미터반경주택수
0,71,(주)감사하는사람들노인복지센터,서울특별시 영등포구 시흥대로 649301호 (대림동),126.902758,37.485598,재가노인복지시설,대림2동,0.469712,0.363636,0.000000,0.003863,0.020134
1,72,21세기요양원 노인요양공동생활가정,서울특별시 영등포구 도신로69 4층층 (도림동),126.900090,37.506351,노인의료복지시설,도림동,0.469712,0.272727,0.285714,0.008255,0.328859
2,52,Vine Center,서울특별시 영등포구 양평로12가길 8 (당산동6가),126.900249,37.536375,한부모가족복지시설,당산6동,0.469712,0.181818,0.142857,0.034537,0.006711
3,76,가온실버복지센터,서울특별시 영등포구 여의대방로 379 8층 14-1호 (여의도동 제일빌딩),126.930932,37.519141,재가노인복지시설,여의동,0.208719,0.000000,0.000000,0.001574,0.000000
4,96,가족사랑1,서울특별시 영등포구 도신로204 4층 (신길동),126.913932,37.511183,노인의료복지시설,신길동,0.208719,0.727273,0.285714,0.009559,0.624161


In [13]:
norm2_df_dBox = norm_df_dBox.copy()
# 정규화된 데이터프레임을 복사하며 저장된 이전 index에 대한 정보인 index열 삭제
norm2_df_dBox = norm2_df_dBox.drop(['index'], axis=1)
# 가중치와 각 변수의 정규화된 값을 곱한 후 다시 저장
for i in range(0, len_weight):
    name = col_name[i]
    norm2_df_dBox[name] = norm2_df_dBox[name] * weight[i] 

# '시설명','시설주소','경도','위도','시설종류상세명','동이름' 컬럼명을 index로 지정
norm2_df_dBox.set_index(['시설명','시설주소','경도','위도','시설종류상세명','동이름'], inplace=True)
norm2_df_dBox.head()

,,,,,,일인가구비율,cctv개수,건물가점,파출소거리역수값,백미터반경주택수
시설명,시설주소,경도,위도,시설종류상세명,동이름,,,,,
(주)감사하는사람들노인복지센터,서울특별시 영등포구 시흥대로 649301호 (대림동),126.902758,37.485598,재가노인복지시설,대림2동,0.086090,0.044706,0.000000,0.000914,0.007983
21세기요양원 노인요양공동생활가정,서울특별시 영등포구 도신로69 4층층 (도림동),126.900090,37.506351,노인의료복지시설,도림동,0.086090,0.033530,0.017349,0.001953,0.130397
Vine Center,서울특별시 영등포구 양평로12가길 8 (당산동6가),126.900249,37.536375,한부모가족복지시설,당산6동,0.086090,0.022353,0.008674,0.008169,0.002661
가온실버복지센터,서울특별시 영등포구 여의대방로 379 8층 14-1호 (여의도동 제일빌딩),126.930932,37.519141,재가노인복지시설,여의동,0.038255,0.000000,0.000000,0.000372,0.000000
가족사랑1,서울특별시 영등포구 도신로204 4층 (신길동),126.913932,37.511183,노인의료복지시설,신길동,0.038255,0.089412,0.017349,0.002261,0.247487


#### 각 시설별로 '일인가구비율', 'cctv개수', '건물가점', '파출소거리역수값', '백미터반경주택수'에 대한 점수의 합 구하기

In [14]:
norm2_df_dBox['sum'] = norm2_df_dBox.iloc[:, 0:5].sum(1)

norm2_df_dBox.reset_index(inplace=True)

#### 시설의 총점을 바탕으로 순위를 매겨 총점이 높은 순으로 정렬

In [15]:
# 순위 계산하기
norm2_df_dBox['순위'] = norm2_df_dBox['sum'].rank(method='dense', ascending=False).astype(int)

# 총점 점수가 높은 순서대로 시설명에 대한 정보를 정렬하기
final = norm2_df_dBox.sort_values(by='순위')
final.reset_index(drop=True, inplace=True)

#### 상위 20개의 시설명 출력, 결과 확인하기 

In [16]:
final.head(20)

,시설명,시설주소,경도,위도,시설종류상세명,동이름,일인가구비율,cctv개수,건물가점,파출소거리역수값,백미터반경주택수,sum,순위
0,영등포본동 아이랜드 (영등포1호점 우리동네키움센터),서울특별시 영등포구 신길로60길 31-2 (신길동),126.913796,37.513506,아동복지시설,영등포동,0.183283,0.067059,0.017349,0.162279,0.396512,0.826483,1
1,신길3동주민센터,-,126.907744,37.507269,-,영등포동,0.183283,0.089412,0.034698,0.001121,0.335305,0.643820,2
2,도림동자치회관,-,126.908024,37.509951,-,대림2동,0.086090,0.111765,0.034698,0.002737,0.351272,0.586563,3
3,사르넬리지역아동센터,서울특별시 영등포구 대림로22길 40,126.904771,37.495636,아동복지시설,대림2동,0.086090,0.111765,0.017349,0.004373,0.322000,0.541577,4
4,남부장애인공동생활가정1호,서울특별시 영등포구 도신로48길 9-13501호 (신길동 다인주택)(신길동 다인주택),126.913524,37.510440,장애인거주시설,대림2동,0.086090,0.089412,0.026023,0.002513,0.327322,0.531361,5
5,푸른공부방지역아동센터,서울특별시 영등포구 양산로 138-1 (당산동1가),126.898287,37.523607,아동복지시설,영등포동,0.183283,0.078236,0.034698,0.004295,0.226198,0.526710,6
6,양평1동주민센터,-,126.888228,37.523646,-,영등포동,0.183283,0.033530,0.052047,0.011401,0.244826,0.525087,7
7,대림1동주민센터,-,126.905948,37.495462,-,대림2동,0.086090,0.089412,0.034698,0.002261,0.303372,0.515833,8
8,사회적협동조합 가가호호돌봄센터,서울특별시 영등포구 영등포로64길 311층 (신길동)(신길동),126.913760,37.512127,재가노인복지시설,대림2동,0.086090,0.044706,0.008674,0.002211,0.372562,0.514244,9
9,어버이사랑노인의료복지센터1,서울특별시 영등포구 도림로318 4층 (신길동),126.902256,37.505957,노인의료복지시설,영등포동,0.183283,0.089412,0.008674,0.002076,0.226198,0.509644,10


#### 최종 결과 CSV 파일쓰기 

In [17]:
final.to_csv('dBox결과_총점.csv', index=False, sep=',', encoding='cp949')